<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=95f825b1cf6f34e78787c976f6fd56758389424255297d3130c4e3c7b49884ae
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 09:23:26
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -6.24 K (-0.04%)
Current PnL: -18.18 L (-12.23%)
CY Booked + Current PnL: -6.74 L (-4.53%)
-------------------
Total profit:  1.67 L
Total loss:  -19.85 L
-------------------
Total Booked + Current PnL: 20.24 L (16.63%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.09%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.05 L (59.43%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,15.30,68.0,H-MC,3.67,199080.0,34410.0,8919.0,-0.30,20.90,4.48,26.31,89.0,3.86,1.41,30.55,X40N,NTT,AC
77,TTKPRESTIG,770.00,99.57,45.0,M-SC,3.08,85995.0,-14782.0,14877.0,-0.05,-14.67,17.30,0.09,245.0,-0.99,0.61,12.98,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,63.0,H-MC,9.26,111406.0,10902.0,18204.0,-0.35,10.85,16.34,28.96,79.0,0.60,0.79,48.50,MH,ATH,METALS
84,VOLTAS,1530.00,-1.08,49.0,H-MC,2.87,207645.0,15903.0,21865.0,0.59,8.29,10.53,19.69,99.0,0.73,1.47,15.66,XY25,NTT,AC
57,RELIANCE,1533.00,-13.29,48.0,H-LC,4.59,216887.0,6521.0,22253.0,-0.02,3.10,10.26,13.68,37.0,0.29,1.53,20.14,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3937.84,62.0,M-SC,5.42,125823.0,-21545.0,73355.0,6.22,-14.62,58.30,35.16,236.0,-0.29,0.89,28.52,XY24,NTT,AUTO
67,SONACOMS,1006.00,-39.60,43.0,M-SC,11.64,75119.0,-26040.0,100930.0,2.06,-25.74,134.36,74.03,202.0,-0.26,0.53,5.92,AR,BTT,AUTO
55,RAJOOENG,143.10,-37.40,37.0,H-SC,12.27,94890.0,-7610.0,48214.0,1.57,-7.42,50.81,39.61,114.0,-0.16,0.67,1.57,AR,ATH,MISC
73,TATAMOTORS,1065.00,-18.87,55.0,M-LC,2.33,256659.0,-19864.0,129946.0,1.55,-7.18,50.63,39.81,54.0,-0.15,1.82,23.01,XY24,NTT,AUTO
33,HONAUT,58357.33,-20.78,50.0,H-SC,2.15,111630.0,-16308.0,63439.0,0.76,-12.75,56.83,36.84,143.0,-0.26,0.79,14.28,X40N,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1225.36,75.0,M-SC,8.06,180173.0,-970.0,46431.0,-2.12,-0.54,25.77,25.10,235.0,-0.02,1.27,44.39,XY24,NTT,HEALTHCARE
8,AWL,485.0,-64.07,59.0,H-MC,3.66,255749.0,-46002.0,207438.0,-1.22,-15.25,81.11,53.50,116.0,-0.22,1.81,13.59,XY24,NTT,FMCG
45,KANSAINER,340.0,-67.42,58.0,H-SC,0.89,225000.0,-44667.0,81000.0,-1.19,-16.56,36.00,13.47,138.0,-0.55,1.59,13.80,XY24,NTT,PAINTS
79,UNIONBANK,163.0,-13.18,63.0,M-LC,7.06,157827.0,16987.0,27509.0,-1.06,12.06,17.43,31.59,66.0,0.62,1.12,41.74,XY24,NTT,BANKS
76,TRIDENT,48.0,-4.48,63.0,M-SC,1.39,74248.0,-18006.0,44311.0,-0.96,-19.52,59.68,28.51,224.0,-0.41,0.53,27.05,XR,NTT,TEXTILES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1225.36,75.0,M-SC,8.06,180173.0,-970.0,46431.0,-2.12,-0.54,25.77,25.10,235.0,-0.02,1.27,44.39,XY24,NTT,HEALTHCARE
29,HAVELLS,2069.16,-0.51,55.0,H-MC,2.04,246792.0,-939.0,75987.0,0.04,-0.38,30.79,30.30,92.0,-0.01,1.75,12.86,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.15,54.0,H-MC,3.79,202125.0,1305.0,67469.0,-0.44,0.65,33.38,34.25,80.0,0.02,1.43,5.88,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,99.57,45.0,M-SC,3.08,85995.0,-14782.0,14877.0,-0.05,-14.67,17.30,0.09,245.0,-0.99,0.61,12.98,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,91.37,48.0,H-SC,9.28,120771.0,-21708.0,41231.0,-0.09,-15.24,34.14,13.70,163.0,-0.53,0.85,42.54,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,58.0,H-SC,0.89,225000.0,-44667.0,81000.0,-1.19,-16.56,36.00,13.47,138.0,-0.55,1.59,13.80,XY24,NTT,PAINTS
18,CERA,9475.0,-20.76,43.0,H-SC,2.07,144486.0,-31417.0,73442.0,-0.32,-17.86,50.83,23.89,149.0,-0.43,1.02,24.78,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-21.02,47.0,H-MC,7.02,106742.0,-25093.0,67322.0,0.20,-19.03,63.07,32.03,98.0,-0.37,0.76,19.56,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.0,15.3,68.0,H-MC,3.67,199080.0,34410.0,8919.0,-0.3,20.9,4.48,26.31,89.0,3.86,1.41,30.55,X40N,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,48.36,68.0,M-MC,8.90,233624.0,8662.0,158981.0,0.13,3.85,68.05,74.53,192.0,0.05,1.65,37.35,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,15.30,68.0,H-MC,3.67,199080.0,34410.0,8919.0,-0.30,20.90,4.48,26.31,89.0,3.86,1.41,30.55,X40N,NTT,AC
17,CAMS,5211.76,-0.73,60.0,H-SC,2.57,112980.0,10976.0,32945.0,0.52,10.76,29.16,43.06,122.0,0.33,0.80,29.72,X40N,ATH,MISC
84,VOLTAS,1530.00,-1.08,49.0,H-MC,2.87,207645.0,15903.0,21865.0,0.59,8.29,10.53,19.69,99.0,0.73,1.47,15.66,XY25,NTT,AC
23,DABUR,735.00,-6.90,41.0,H-MC,6.93,251982.0,3914.0,104497.0,-0.48,1.58,41.47,43.70,102.0,0.04,1.78,16.25,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,63.0,H-MC,9.26,111406.0,10902.0,18204.0,-0.35,10.85,16.34,28.96,79.0,0.60,0.79,48.50,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.92,49.0,M-SC,1.15,99765.0,8266.0,70484.0,0.08,9.03,70.65,86.07,223.0,0.12,0.71,44.88,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,67.0,H-MC,6.97,185375.0,5567.0,106850.0,-0.00,3.10,57.64,62.52,88.0,0.05,1.31,38.24,XR,NTT,BANKS
86,WIPRO,420.00,-11.52,50.0,M-LC,5.67,155732.0,4787.0,104667.0,0.34,3.17,67.21,72.51,53.0,0.05,1.10,9.38,XR,NTT,IT
37,INDIAMART,4850.92,-53.71,31.0,H-SC,11.04,129198.0,5862.0,127906.0,-0.47,4.75,99.00,108.45,119.0,0.05,0.91,28.17,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-17.78,38.0,H-LC,7.13,296351.0,-19491.0,132143.0,0.12,-6.17,44.59,35.67,5.0,-0.15,2.10,6.83,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.51,41.0,H-LC,9.90,225342.0,-16582.0,79861.0,-0.32,-6.85,35.44,26.15,8.0,-0.21,1.59,6.49,X40,ATH,IT
5,ASIANPAINT,3465.66,-13.84,43.0,H-LC,7.66,217316.0,-34452.0,87665.0,-0.53,-13.68,40.34,21.13,27.0,-0.39,1.54,16.89,X40,ATH,PAINTS
42,ITC,452.00,-38.34,44.0,H-LC,2.02,198436.0,-1702.0,22582.0,-0.28,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
31,HINDUNILVR,2922.00,-8.54,45.0,H-LC,10.37,256750.0,-313.0,35457.0,-0.18,-0.12,13.81,13.67,24.0,-0.01,1.82,20.26,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.34,50.0,H-LC,1.48,237082.0,-9069.0,88266.0,0.16,-3.68,37.23,32.18,16.0,-0.10,1.68,31.03,X200,ATH,IT
42,ITC,452.0,-38.34,44.0,H-LC,2.02,198436.0,-1702.0,22582.0,-0.28,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
65,SIEMENS,4671.5,1.24,56.0,H-LC,3.32,162750.0,-23345.0,70829.0,0.35,-12.54,43.52,25.51,15.0,-0.33,1.15,19.93,AR,ATH,ELECTRICAL
51,NESTLEIND,1377.0,-10.26,47.0,H-LC,4.14,276120.0,10694.0,46084.0,-0.70,4.03,16.69,21.40,11.0,0.23,1.95,10.94,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.29,48.0,H-LC,4.59,216887.0,6521.0,22253.0,-0.02,3.10,10.26,13.68,37.0,0.29,1.53,20.14,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.40,45.0,H-LC,12.15,289182.0,-56774.0,131636.0,0.08,-16.41,45.52,21.64,1.0,-0.43,2.05,2.44,X40,ATH,IT
42,ITC,452.00,-38.34,44.0,H-LC,2.02,198436.0,-1702.0,22582.0,-0.28,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.37,54.0,H-LC,7.52,229534.0,-4610.0,53229.0,-0.40,-1.97,23.19,20.77,86.0,-0.09,1.62,4.81,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.51,41.0,H-LC,9.90,225342.0,-16582.0,79861.0,-0.32,-6.85,35.44,26.15,8.0,-0.21,1.59,6.49,X40,ATH,IT
83,VBL,671.64,-17.78,38.0,H-LC,7.13,296351.0,-19491.0,132143.0,0.12,-6.17,44.59,35.67,5.0,-0.15,2.10,6.83,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.24,56.0,H-LC,3.32,162750.0,-23345.0,70829.0,0.35,-12.54,43.52,25.51,15.0,-0.33,1.15,19.93,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.34,44.0,H-LC,2.02,198436.0,-1702.0,22582.0,-0.28,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,45.0,H-LC,5.36,211336.0,6836.0,34575.0,0.03,3.34,16.36,20.25,10.0,0.20,1.49,12.37,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-13.29,48.0,H-LC,4.59,216887.0,6521.0,22253.0,-0.02,3.10,10.26,13.68,37.0,0.29,1.53,20.14,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-13.84,43.0,H-LC,7.66,217316.0,-34452.0,87665.0,-0.53,-13.68,40.34,21.13,27.0,-0.39,1.54,16.89,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.40,45.0,H-LC,12.15,289182.0,-56774.0,131636.0,0.08,-16.41,45.52,21.64,1.0,-0.43,2.05,2.44,X40,ATH,IT
40,INFY,2275.00,-17.44,51.0,H-LC,8.14,320991.0,8103.0,163577.0,0.50,2.59,50.96,54.87,3.0,0.05,2.27,9.41,X40,BTT,IT
42,ITC,452.00,-38.34,44.0,H-LC,2.02,198436.0,-1702.0,22582.0,-0.28,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
83,VBL,671.64,-17.78,38.0,H-LC,7.13,296351.0,-19491.0,132143.0,0.12,-6.17,44.59,35.67,5.0,-0.15,2.10,6.83,X40N,ATH,FMCG
1,ABB,7934.00,-38.41,60.0,H-LC,8.98,257534.0,-4085.0,123307.0,0.05,-1.56,47.88,45.57,7.0,-0.03,1.82,9.58,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7088.89,68.0,L-SC,5.36,78882.0,-14928.0,91543.0,0.32,-15.91,116.05,81.67,269.0,-0.16,0.56,52.76,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.27,82541.0,-31008.0,71060.0,0.07,-27.31,86.09,35.27,268.0,-0.44,0.58,102.02,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-173.86,65.0,M-SC,4.52,84708.0,-17502.0,110891.0,0.01,-17.12,130.91,91.37,208.0,-0.16,0.60,37.51,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-6.19,42.0,H-SC,12.05,87333.0,-13630.0,108162.0,0.15,-13.50,123.85,93.63,148.0,-0.13,0.62,30.08,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.67,94050.0,-3930.0,25535.0,-0.43,-4.01,27.15,22.05,152.0,-0.15,0.67,36.71,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1225.36,75.0,M-SC,8.06,180173.0,-970.0,46431.0,-2.12,-0.54,25.77,25.10,235.0,-0.02,1.27,44.39,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3937.84,62.0,M-SC,5.42,125823.0,-21545.0,73355.0,6.22,-14.62,58.30,35.16,236.0,-0.29,0.89,28.52,XY24,NTT,AUTO
61,SAMMAANCAP,326.00,-173.86,65.0,M-SC,4.52,84708.0,-17502.0,110891.0,0.01,-17.12,130.91,91.37,208.0,-0.16,0.60,37.51,XY25,NTT,FINANCE
71,TANLA,1963.11,-16.63,69.0,H-SC,14.34,223992.0,-46769.0,376710.0,-0.43,-17.27,168.18,121.86,133.0,-0.12,1.58,75.22,AR,ATH,IT
81,VAIBHAVGBL,521.00,62.53,65.0,H-SC,6.01,141096.0,-41679.0,152751.0,-0.21,-22.80,108.26,60.77,125.0,-0.27,1.00,26.37,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.34
1,25,43.64
2,50,74.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.06
LC    32.41
MC    23.55
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.37
X40      14.70
X40N     11.43
XR       11.28
XY25     10.05
AR        8.40
OX40N     7.83
X200      1.68
MH        1.65
X5K       1.41
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.36
H-LC    25.93
H-MC    20.62
M-SC    13.20
M-LC     5.45
M-MC     2.63
L-SC     1.50
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.94,-4.17,38.55
IT,12.94,-14.90,77.55
FINANCE,10.04,-15.69,66.16
MISC,7.07,-15.01,78.94
BANKS,6.07,-16.38,77.61
PAINTS,5.62,-16.20,33.52
ELECTRICAL,5.51,-5.74,42.84
INSURANCE,3.68,-6.28,46.01
AC,3.58,4.17,11.88


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3088551.0,22
XR,1317902.0,14
AR,1142957.0,9
X40,770667.0,10
X40N,588500.0,9
OX40N,541016.0,10
XY25,418633.0,7
SR,248973.0,2
X5K,117770.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3643653.0,29
M-SC,1382818.0,17
H-MC,1191709.0,15
H-LC,1153569.0,15
M-LC,379892.0,4
M-MC,305857.0,2
L-SC,248328.0,3
L-MC,60706.0,1
L-LC,38228.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1223694.0      6
M-SC       XY24       791429.0      7
H-SC       AR         785493.0      5
           XR         768281.0      7
H-MC       XY24       548011.0      4
H-LC       X40        485321.0      5
M-MC       XY24       305857.0      2
H-SC       X40N       296135.0      4
           OX40N      257756.0      4
           SR         248973.0      2
H-LC       X40N       219947.0      3
H-MC       X40        213184.0      4
H-LC       AR         194136.0      2
H-MC       XY25       165720.0      2
M-SC       AR         163328.0      2
L-SC       XR         162603.0      2
M-LC       XY24       157455.0      2
M-SC       OX40N      130213.0      4
M-LC       X5K        117770.0      1
M-SC       XR         114795.0      2
           XY25       110891.0      1
H-MC       XR         106850.0      1
M-LC       XR         104667.0      1
H-LC       XY25       103794.0      3
           X200        88266.0      1
L-SC       OX40N       85725.0      1
H-MC       X40N        72418.0      2
M-SC       X40         72162.0      1
H-MC       OX40N       67322.0      1
H-SC       MH          63321.0      1
H-LC       XY24        62105.0      1
L-MC       XR          60706.0      1
L-LC       XY25        38228.0      1
H-MC       MH          18204.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
